# Container script for pilot training:

Using deterministic CLS tokens

Notes:
- ran multiple runs of layer 10 due to saving errors. They look different, probably because of random initialization of the (deterministic) CLS generator

In [1]:
import sys
import torch

from src.model.setup import image_model_setup
from src.data.activity_dataset import OnlineLayerInputDataset
from src.model.CLS_token_probing import ModuleSpecificDecoder


In [2]:
mode = 'replace'
device = 'cuda'

In [3]:
torch.set_float32_matmul_precision('high')

model_name = "facebook/dinov2-base"
dataset_name = "../workspace/temp_dataset_subsample"

# model for train data generate
model, image_datasets, _ = image_model_setup(model_name, dataset_name, 1000)

# second copy of the model: used for e
model_analysis, _, _ = image_model_setup(model_name, dataset_name, 1000)
model_analysis.to(device)
model_analysis.model.eval()

Loading weights:   0%|          | 0/223 [00:00<?, ?it/s]

Dinov2ForImageClassification LOAD REPORT from: facebook/dinov2-base
Key               | Status  | 
------------------+---------+-
classifier.weight | MISSING | 
classifier.bias   | MISSING | 

Notes:
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


Loading weights:   0%|          | 0/223 [00:00<?, ?it/s]

Dinov2ForImageClassification LOAD REPORT from: facebook/dinov2-base
Key               | Status  | 
------------------+---------+-
classifier.weight | MISSING | 
classifier.bias   | MISSING | 

Notes:
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


Dinov2ForImageClassification(
  (dinov2): Dinov2Model(
    (embeddings): Dinov2Embeddings(
      (patch_embeddings): Dinov2PatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(14, 14), stride=(14, 14))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): Dinov2Encoder(
      (layer): ModuleList(
        (0-11): 12 x Dinov2Layer(
          (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (attention): Dinov2Attention(
            (attention): Dinov2SelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
            )
            (output): Dinov2SelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (layer_scale1): Dinov2Laye

In [4]:
from src.model.CLS_token_probing import DeterministicCLSGenerator

The previous training script doesn't work with modules that can have configurable sub-modules. Quick hack to accomplish this in the pilot.

In [5]:
import torch
import wandb
from torch.optim.lr_scheduler import CosineAnnealingLR

import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint, LearningRateMonitor


def train_module_decoder(decoder_model, base_modules,
                         train_dataloader, val_dataloader,
                         max_epochs=5, patience=5,
                         accelerator="auto", devices="auto",
                         **trainer_kwargs):
    """
    Trains a ModuleSpecificDecoder using pytorch_lightning. Early stops on validation loss.
    Loads the best model weights at the end.

    Args:
        decoder_model: An ModuleSpecificDecoder instance
        base_modules: The modules that are being decoded
        train_dataloader: PyTorch DataLoader for training set.
        val_dataloader: PyTorch DataLoader for validation set.
        max_epochs: Maximum epochs to train.
        patience: Patience for early stopping on val_loss.
        accelerator: Accelerator for training ("cpu", "gpu", "auto", etc.).
        devices: Devices specification ("auto", int, etc.).
        **trainer_kwargs: Additional kwargs for Trainer.
    Returns:
        Trained (best weights) model.
    """

    # Early stopping on validation loss
    early_stop_cb = EarlyStopping(
        monitor="val/accuracy",
        patience=patience,
        verbose=True,
        mode="max"
    )

    # ModelCheckpoint to save the best model
    checkpoint_cb = ModelCheckpoint(
        monitor="val/accuracy",
        save_top_k=1,
        mode="max",
        save_last=True,
        filename="best"
    )

    lr_monitor = LearningRateMonitor(logging_interval='step')

    wandb_logger = WandbLogger(project="middle_decoders", log_model=True)

    trainer = pl.Trainer(
        max_epochs=max_epochs,
        logger=wandb_logger,
        callbacks=[early_stop_cb, checkpoint_cb, lr_monitor],
        accelerator=accelerator,
        devices=devices,
        **trainer_kwargs,
    )

    trainer_model = TrainingWrapper_Decoder(decoder_model, base_modules, num_epochs=max_epochs)

    trainer.fit(trainer_model, train_dataloader, val_dataloader)

    # Load the best checkpoint, if available
    best_model_path = checkpoint_cb.best_model_path
    if best_model_path:
        decoder_model = type(decoder_model).load_from_checkpoint(best_model_path, generator_class=DeterministicCLSGenerator)

    wandb.finish()

    return decoder_model


class TrainingWrapper_Decoder(pl.LightningModule):
    """Surrogate Module to use for training the Module decoders 

        Manages base_module dependence, training hyperparameters
    """
    def __init__(self, decoder=None, base_modules=None, lr=1E-3, num_epochs=3):
        super().__init__()
        self.decoder = decoder
        self.base_modules = base_modules
        self.lr = lr
        self.num_epochs = num_epochs

        # Are we decoding single layers or multiple layers?
        self.multiple_decoder = isinstance(base_modules, dict)

        if self.multiple_decoder:
            to_freeze = self.base_modules.values()
        else:
            to_freeze = [self.base_modules]

        # freeze the parameters of the base module
        for module in to_freeze:
            for param in module.parameters():
                param.requires_grad = False

            module.eval()

    def forward(self, input_tokens):
        if not self.multiple_decoder:
            # the module is a single decoder module
            inputs = list(input_tokens.items())[0][1]
            return self.decoder.forward(inputs, self.base_modules)
        else:
            input_module_pairs = {key: (input_tokens[key], self.base_modules[key])
                                  for key in self.base_modules.keys()
                                  }
            return self.decoder.forward(input_module_pairs)

    def training_step(self, batch, batch_idx):
        """
        Assumes batch = (input_tokens, targets)
        - input_tokens: (B, L, D) float tensor
        - targets: (B,) class indices
        """
        targets = batch.pop('label')
        input_tokens = batch

        logits = self.forward(input_tokens)
        loss = self.decoder.loss_fn(logits, targets)

        self.log("train/loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        targets = batch.pop('label')
        input_tokens = batch

        logits = self.forward(input_tokens)
        loss = self.decoder.loss_fn(logits, targets)
        acc = (logits.argmax(-1) == targets).float().mean()
        self.log("val/loss", loss, prog_bar=True)
        self.log("val/accuracy", acc, prog_bar=True)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        # T_max is the number of epochs to reach the minimum LR
        scheduler = CosineAnnealingLR(optimizer, T_max=self.num_epochs*3125, eta_min=1e-6)

        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": scheduler,
                "interval": "step",
            },
        }

    def on_save_checkpoint(self, checkpoint):
        """ Strip the base_module weights from the checkpoint """
        checkpoint['state_dict'] = self.decoder.state_dict()

In [6]:
i = 10

layer_name = f'dinov2.encoder.layer.{i}'

ds_train = OnlineLayerInputDataset(model, layer_name, image_datasets['train'],
                               device=device)
ds_validation = OnlineLayerInputDataset(model, layer_name, image_datasets['validation'],
                                    device=device)

probe = ModuleSpecificDecoder(mode=mode, generator_class=DeterministicCLSGenerator)
base_module = model_analysis.module_dict[layer_name]
model_out = train_module_decoder(probe, base_module, ds_train, ds_validation)
model_out.save_pretrained(f'outs_{mode}/layer{i}_probe_{mode}')

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
💡 Tip: For seamless cloud logging and experiment tracking, try installing [litlogger](https://pypi.org/project/litlogger/) to enable LitLogger, which logs metrics and artifacts automatically to the Lightning Experiments platform.
wandb: WARNING The anonymous setting has no effect and will be removed in a future version.
wandb: [wandb.login()] Loaded credentials for https://api.wandb.ai from WANDB_API_KEY.
wandb: Currently logged in as: lrast to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name         ┃ Type                  ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ decoder      │ ModuleSpecificDecoder │  769 K │ train │     0 │
│ 1 │ base_modules │ Dinov2Layer           │  7.1 M │ eval  │     0 │
└───┴──────────────┴───────────────────────┴────────┴───────┴───────┘

Trainable params: 769 K                                                                                            
Non-trainable params: 7.1 M                                                                                        
Total params: 7.9 M                                                                                                
Total estimated model params size (MB): 31                                                                         
Modules in train mode: 4                                                                                           
Modules in eval mode: 18                                                                                           
Total FLOPs: 0

Output()

/usr/local/lib/python3.11/dist-packages/pytorch_lightning/loops/fit_loop.py:534: Found 18 module(s) in eval mode at
the start of training. This may lead to unexpected behavior during training. If this is intentional, you can ignore
this warning.

/usr/local/lib/python3.11/dist-packages/PIL/TiffImagePlugin.py:870: UserWarning: Corrupt EXIF data.  Expecting to 
read 2 bytes but only got 0. 
  warnings.warn(str(msg))

Metric val/accuracy improved. New best score: 0.686
Metric val/accuracy improved by 0.051 >= min_delta = 0.0. New best score: 0.738
Metric val/accuracy improved by 0.020 >= min_delta = 0.0. New best score: 0.758
Metric val/accuracy improved by 0.011 >= min_delta = 0.0. New best score: 0.769
IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

Metric val/accuracy improved by 0.004 >= min_delta = 0.0. New best score: 0.773
`Trainer.fit` stopped: `max_epochs=5` reached.


/usr/local/lib/python3.11/dist-packages/lightning_fabric/utilities/cloud_io.py:73: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.


epoch,▁▁▁▁▁▁▁▁▁▃▃▃▃▃▅▅▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆█████████
lr-Adam,████████▇▇▇▇▇▆▆▅▅▅▅▅▅▄▄▃▃▃▃▃▂▂▂▂▁▁▁▁▁▁▁▁
train/loss,█▇▇▄▄▃▂▂▂▁▂▂▂▂▂▂▂▂▂▂▂▁▁▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
trainer/global_step,▁▁▁▁▁▂▂▂▂▂▂▂▃▃▃▄▄▄▄▄▄▄▅▅▅▆▆▆▆▆▆▆▆▆▆▇▇▇▇█
val/accuracy,▁▅▇██
val/loss,█▄▂▁▁
epoch,4
lr-Adam,0.0
train/loss,0.59645
trainer/global_step,15624
val/accuracy,0.77288


In [ ]:
layer_inds = [8, 9, 11]

for i in layer_inds:
    layer_name = f'dinov2.encoder.layer.{i}'

    ds_train = OnlineLayerInputDataset(model, layer_name, image_datasets['train'],
                                       device=device)
    ds_validation = OnlineLayerInputDataset(model, layer_name, image_datasets['validation'],
                                            device=device)

    probe = ModuleSpecificDecoder(mode=mode, generator_class=DeterministicCLSGenerator)

    base_module = model_analysis.module_dict[layer_name]

    model_out = train_module_decoder(probe, base_module, ds_train, ds_validation)
    model_out.save_pretrained(f'outs_{mode}/layer{i}_probe_{mode}')

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
💡 Tip: For seamless cloud logging and experiment tracking, try installing [litlogger](https://pypi.org/project/litlogger/) to enable LitLogger, which logs metrics and artifacts automatically to the Lightning Experiments platform.
wandb: WARNING The anonymous setting has no effect and will be removed in a future version.
wandb: [wandb.login()] Loaded credentials for https://api.wandb.ai from WANDB_API_KEY.
wandb: Currently logged in as: lrast to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name         ┃ Type                  ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ decoder      │ ModuleSpecificDecoder │  769 K │ train │     0 │
│ 1 │ base_modules │ Dinov2Layer           │  7.1 M │ eval  │     0 │
└───┴──────────────┴───────────────────────┴────────┴───────┴───────┘

Trainable params: 769 K                                                                                            
Non-trainable params: 7.1 M                                                                                        
Total params: 7.9 M                                                                                                
Total estimated model params size (MB): 31                                                                         
Modules in train mode: 4                                                                                           
Modules in eval mode: 18                                                                                           
Total FLOPs: 0

Output()

/usr/local/lib/python3.11/dist-packages/pytorch_lightning/loops/fit_loop.py:534: Found 18 module(s) in eval mode at
the start of training. This may lead to unexpected behavior during training. If this is intentional, you can ignore
this warning.

/usr/local/lib/python3.11/dist-packages/PIL/TiffImagePlugin.py:870: UserWarning: Corrupt EXIF data.  Expecting to 
read 2 bytes but only got 0. 
  warnings.warn(str(msg))

Metric val/accuracy improved. New best score: 0.444
Metric val/accuracy improved by 0.115 >= min_delta = 0.0. New best score: 0.558
IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

Metric val/accuracy improved by 0.040 >= min_delta = 0.0. New best score: 0.599
IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

